In [1]:
from services.report_service import get_hour_data_prices

MERCHANDISE_RATE = 'LINKUSDT'
DAYS = 60
HOUR_OBSERVE = 10

data_prices = get_hour_data_prices(MERCHANDISE_RATE, DAYS)

12:38:33 DEBUG
_________________________________________log query_________________________________________
12:38:33 DEBUG
SELECT * FROM DailyTradingJournal_development.candlesticks candlesticks INNER JOIN DailyTradingJournal_development.hour_analytics ON candlesticks.id = hour_analytics.candlestick_id WHERE candlesticks.time_type = 4 AND candlesticks.merchandise_rate_id = 42 ORDER BY candlesticks.date DESC lIMIT 1440;
12:38:33 DEBUG
	                            open   high  close     low     volumn date_database date_with_binane  hour  return_oc  return_hl  candlestick_type  range_type  is_same_btc  continue_by_day  continue_by_hour         day
	date                                                                                                                                                                                                                  
	2024-10-03 10:00:00+07:00  10.75  10.81  10.79  10.720    27670.4    2024-10-03       2024-10-03    10       0.37       0.84      

In [ ]:
from services.report_service import hour_analytic

plt = hour_analytic(data_prices)
plt.show()

In [ ]:
from services.report_service import detail_hour_analytic

plt = detail_hour_analytic(data_prices, HOUR_OBSERVE)
# plt.show()

In [ ]:
import sys
import os
print(sys.path)
print(os.environ["PYTHONPATH"])